In [1]:
# Optimized Biomarker Selection

In [2]:
import pandas as pd
import numpy as np
import gudhi
import os
from sklearn.linear_model import LinearRegression

def Optimized_Biomarker_Selection(
    distance_threshold_offset,
    tumor_types,
    max_patients,
    normal_data_size,
    top_n_files_to_eliminate,
    cancer_types,
    Select_Biomarkers,
    clinical_data_path,  # User-defined path for the clinical data Excel file
    sheet_name,
    feature_vector_path,  # User-defined path for the feature vector CSV file
    output_folder_path,  # User-defined folder path for saving outputs
    required_normal_positives  # User-defined required number of normal positives
):
    # Part 1: Perform TDA on the clinical data and save the results to a CSV file
    def part_1():
        file_path = clinical_data_path
        df_1 = pd.read_excel(file_path, sheet_name=sheet_name)
        df_2 = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Check if Select_Biomarkers exist in the DataFrame
        missing_columns = [col for col in Select_Biomarkers if col not in df_1.columns]
        if missing_columns:
            raise ValueError(f"The following columns are missing in the dataset: {missing_columns}")
        
        df_1 = df_1[Select_Biomarkers]
        df_2 = df_2[Select_Biomarkers]
        
        df_1 = df_1.iloc[:59]
        data_list = []
        
        for index in range(60, max_patients + 1):
            row_to_add = df_2.iloc[[index]]
            df_new = pd.concat([df_1] + [row_to_add] * 1, ignore_index=True)
            
            numeric_cols = df_new.select_dtypes(include=[np.number]).columns
            df_new[numeric_cols] = np.log(df_new[numeric_cols])
            
            column_names = df_new.columns.tolist()
            df_subset = df_new.values
            
            corr_matrix = np.corrcoef(df_subset, rowvar=False)
            corr_matrix_df = pd.DataFrame(corr_matrix, columns=column_names, index=column_names)
            distance_matrix = 1 - np.abs(corr_matrix_df)
            distance_matrix_np = distance_matrix.values
            
            rips_complex = gudhi.RipsComplex(distance_matrix=distance_matrix_np)
            simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
            persistence = simplex_tree.persistence()
            
            finite_persistence = [(dim, (birth, death)) for dim, (birth, death) in persistence if death != float('inf')]
            one_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 1]
            
            if one_dim_holes:
                max_life_range_1D = max(death - birth for birth, death in one_dim_holes)
                avg_birth_1D = np.mean([birth for birth, _ in one_dim_holes])
                avg_death_1D = np.mean([death for _, death in one_dim_holes])
                num_one_dim_holes = len(one_dim_holes)
            else:
                max_life_range_1D = avg_birth_1D = avg_death_1D = num_one_dim_holes = 0
            
            zero_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 0]
            min_life_range_0D = min(death - birth for birth, death in zero_dim_holes)
            max_life_range_0D = max(death - birth for birth, death in zero_dim_holes)
            avg_death_0D = np.mean([death for _, death in zero_dim_holes])
            num_zero_dim_holes = len(zero_dim_holes) + 1
            range_death_0D = max([death for _, death in zero_dim_holes]) - min([death for _, death in zero_dim_holes])
            median_death_0D = np.median([death for _, death in zero_dim_holes])
            std_death_0D = np.std([death for _, death in zero_dim_holes])
            
            data = {
                "0-Dim Hole Min Life Range": [min_life_range_0D],
                "0-Dim Hole Max Life Range": [max_life_range_0D],
                "Range_Death_0D": [range_death_0D],
                "Num_0D_Holes": [num_zero_dim_holes],
                "Median_Death_0D": [median_death_0D],
                "Std_Death_0D": [std_death_0D],
                "Avg_Death_0D": [avg_death_0D],
                "Average Birth (1D)": [avg_birth_1D],
                "Average Death (1D)": [avg_death_1D],
                "1-Dim Hole Max Life Range": [max_life_range_1D],
                "Number of 1-Dimensional Holes": [num_one_dim_holes]
            }
            
            df = pd.DataFrame(data, index=[f"P_{index}"])
            data_list.append(df)
        
        final_df = pd.concat(data_list)
        csv_file_path = feature_vector_path
        
        if os.path.exists(csv_file_path):
            existing_df = pd.read_csv(csv_file_path, index_col=0)
            updated_df = pd.concat([existing_df, final_df])
        else:
            updated_df = final_df
        
        updated_df.to_csv(csv_file_path, float_format="%f", index=True)
        print(f"CSV file 'FeatureVectorCancerDet.csv' updated with rows P_60 to P_{max_patients}.")
        
        additional_columns = ['Patient ID #', 'Cohort', 'AJCC Stage']
        df_clinical = pd.read_excel(file_path, sheet_name=sheet_name)
        df_additional_info = df_clinical[additional_columns].iloc[60:max_patients + 1]
        df_additional_info.index = [f"P_{i}" for i in range(60, max_patients + 1)]
        
        df_combined = pd.concat([df_additional_info, final_df], axis=1)
        df_combined.to_csv(csv_file_path, float_format="%f", index=True)
        print(f"CSV file '{csv_file_path}' has been updated with all the patient information.")
    
    # Part 2: Construct a linear regression model and analyze the data
    def part_2():
        file_path = feature_vector_path
        new_data = pd.read_csv(file_path, header=0)
        
        def construct_linear_regression_model():
            data = new_data.iloc[:normal_data_size]
            X = data[['0-Dim Hole Min Life Range', '0-Dim Hole Max Life Range', 'Median_Death_0D', 'Std_Death_0D']]
            y = data['Avg_Death_0D']
            
            model = LinearRegression()
            model.fit(X, y)
            
            coefficients = model.coef_
            intercept = model.intercept_
            
            print(f"\nRegression Equation: Avg_Death_0D = {coefficients[0]:.4f} * 0-Dim Hole Min Life Range + "
                  f"{coefficients[1]:.4f} * 0-Dim Hole Max Life Range + "
                  f"{coefficients[2]:.4f} * Median_Death_0D + "
                  f"{coefficients[3]:.4f} * Std_Death_0D + "
                  f"{intercept:.4f}")
            
            return model, coefficients, intercept
        
        def predict_and_analyze():
            _, coefficients, intercept = construct_linear_regression_model()
            
            def predict_Avg_Death_0D(row):
                return (coefficients[0] * row['0-Dim Hole Min Life Range'] +
                        coefficients[1] * row['0-Dim Hole Max Life Range'] +
                        coefficients[2] * row['Median_Death_0D'] +
                        coefficients[3] * row['Std_Death_0D'] +
                        intercept)
            
            new_data['Predicted Avg_Death_0D'] = new_data.apply(predict_Avg_Death_0D, axis=1)
            new_data['Difference'] = new_data['Avg_Death_0D'] - new_data['Predicted Avg_Death_0D']
            new_data['Regression Difference Percentage'] = abs(new_data['Difference'] / new_data['Avg_Death_0D']) * 100
            
            Normal_data = new_data.iloc[:normal_data_size]
            reference_point = (Normal_data['Average Birth (1D)'].mean(), Normal_data['Average Death (1D)'].mean())
            
            print(f"\nReference Point (Normal Cohort): {reference_point}")
            
            def euclidean_distance(x1, y1, x2, y2):
                return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            
            new_data['Distance %(1)'] = new_data.apply(
                lambda row: euclidean_distance(row['Average Birth (1D)'], row['Average Death (1D)'], reference_point[0], reference_point[1]) * 100,
                axis=1
            )
            
            columns_to_average = [
                '0-Dim Hole Min Life Range',
                'Range_Death_0D',
                '1-Dim Hole Max Life Range',
                'Average Birth (1D)',
                'Average Death (1D)',
                'Regression Difference Percentage',
                'Distance %(1)'
            ]
            
            averages_list = []
            
            for tumor, row_limit in tumor_types.items():
                filtered_data = new_data[new_data['Cohort'].str.strip().str.lower() == tumor]
                selected_data = filtered_data.iloc[:row_limit]
                selected_data = selected_data[columns_to_average].dropna()
                column_averages = selected_data.mean(axis=0)
                averages_list.append({'Cohort': tumor.capitalize(), **column_averages.to_dict()})
            
            averages_data = pd.DataFrame(averages_list)
            new_data.to_csv(file_path, index=False)
            print("\nUpdated data has been saved back to the CSV file.")
            
            print("\nFinal Averages Data:")
            print(averages_data)
            
            Normal_row = averages_data[averages_data['Cohort'].str.strip().str.lower() == 'normal']
            
            if not Normal_row.empty:
                Normal_distance_threshold = Normal_row['Distance %(1)'].values[0] + distance_threshold_offset
                print(f"\nFinal 'Distance %(1)' Threshold (k_2): {Normal_distance_threshold:.6f}")
            else:
                print("Error: 'Normal' cohort not found in averages_data.")
                return None
            
            threshold = 0
            max_threshold = 1.0
            
            while threshold < max_threshold:
                new_data['Result(Cancer)'] = new_data.apply(
                    lambda row: 'Positive' if sum([
                        row['Regression Difference Percentage'] > threshold,
                        row['Distance %(1)'] > Normal_distance_threshold
                    ]) >= 2 else 'Negative',
                    axis=1
                )
                
                positive_counts = new_data[new_data['Result(Cancer)'] == 'Positive'].groupby('Cohort').size()
                Normal_positive = positive_counts.get('Normal', 0)
                
                if Normal_positive == required_normal_positives:
                    print(f"\nFinal Regression Difference Threshold (k_1): {threshold:.4f}")
                    total_positive_cancers = sum(positive_counts.get(t, 0) for t in cancer_types)
                    print(f"Total Positive Cancers: {total_positive_cancers}")
                    return Normal_distance_threshold
                
                threshold += 0.001
            
            print("\nRequired result for the given threshold is not obtained.")
            return Normal_distance_threshold
        
        Normal_threshold = predict_and_analyze()
        print(f"\nNormal Threshold (k_2): {Normal_threshold}")
        return Normal_threshold
    
    # Part 3: Loop through the biomarkers, leaving one out at a time, and perform TDA on the modified dataset
    def part_3():
        file_path = clinical_data_path
        df_1 = pd.read_excel(file_path, sheet_name=sheet_name)
        df_2 = pd.read_excel(file_path, sheet_name=sheet_name)
        
        additional_columns = ['Patient ID #', 'Cohort', 'AJCC Stage']
        df_additional_info = df_2[additional_columns].iloc[60:max_patients + 1]
        df_additional_info.index = [f"P_{i}" for i in range(60, max_patients + 1)]
        
        biomarker_file_map = {}
        file_biomarker_map = {}
        
        for i, biomarker_to_leave in enumerate(Select_Biomarkers):
            selected_columns = [b for b in Select_Biomarkers if b != biomarker_to_leave]
            
            df_1_subset = df_1[selected_columns].iloc[:59]
            data_list = []
            
            for index in range(60, max_patients + 1):
                row_to_add = df_2[selected_columns].iloc[[index]]
                df_new = pd.concat([df_1_subset] + [row_to_add] * 1, ignore_index=True)
                
                numeric_cols = df_new.select_dtypes(include=[np.number]).columns
                df_new[numeric_cols] = np.log(df_new[numeric_cols])
                
                column_names = df_new.columns.tolist()
                df_subset = df_new.values
                
                corr_matrix = np.corrcoef(df_subset, rowvar=False)
                corr_matrix_df = pd.DataFrame(corr_matrix, columns=column_names, index=column_names)
                distance_matrix = 1 - np.abs(corr_matrix_df)
                
                rips_complex = gudhi.RipsComplex(distance_matrix=distance_matrix.values)
                simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
                persistence = simplex_tree.persistence()
                
                finite_persistence = [(dim, (birth, death)) for dim, (birth, death) in persistence if death != float('inf')]
                one_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 1]
                
                if one_dim_holes:
                    max_life_range_1D = max(death - birth for birth, death in one_dim_holes)
                    avg_birth_1D = np.mean([birth for birth, _ in one_dim_holes])
                    avg_death_1D = np.mean([death for _, death in one_dim_holes])
                    num_one_dim_holes = len(one_dim_holes)
                else:
                    max_life_range_1D = avg_birth_1D = avg_death_1D = num_one_dim_holes = 0
                
                zero_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 0]
                min_life_range_0D = min(death - birth for birth, death in zero_dim_holes)
                max_life_range_0D = max(death - birth for birth, death in zero_dim_holes)
                avg_death_0D = np.mean([death for _, death in zero_dim_holes])
                num_zero_dim_holes = len(zero_dim_holes) + 1
                range_death_0D = max([death for _, death in zero_dim_holes]) - min([death for _, death in zero_dim_holes])
                median_death_0D = np.median([death for _, death in zero_dim_holes])
                std_death_0D = np.std([death for _, death in zero_dim_holes])
                
                data = {
                    "0-Dim Hole Min Life Range": [min_life_range_0D],
                    "0-Dim Hole Max Life Range": [max_life_range_0D],
                    "Range_Death_0D": [range_death_0D],
                    "Num_0D_Holes": [num_zero_dim_holes],
                    "Median_Death_0D": [median_death_0D],
                    "Std_Death_0D": [std_death_0D],
                    "Avg_Death_0D": [avg_death_0D],
                    "Average Birth (1D)": [avg_birth_1D],
                    "Average Death (1D)": [avg_death_1D],
                    "1-Dim Hole Max Life Range": [max_life_range_1D],
                    "Number of 1-Dimensional Holes": [num_one_dim_holes]
                }
                
                df = pd.DataFrame(data, index=[f"P_{index}"])
                data_list.append(df)
            
            final_df = pd.concat(data_list)
            final_df = pd.concat([df_additional_info, final_df], axis=1)
            
            csv_file_path = os.path.join(output_folder_path, f"FeatureVectorCancerDet{i + 1}.csv")
            final_df.to_csv(csv_file_path, float_format="%f", index=True)
            
            biomarker_file_map[f"FeatureVectorCancerDet{i + 1}.csv"] = biomarker_to_leave
            file_biomarker_map[csv_file_path] = biomarker_to_leave
            
            print(f"CSV file 'FeatureVectorCancerDet{i + 1}.csv' created by leaving biomarker: {biomarker_to_leave}")
        
        print("\nMapping of biomarkers to their respective CSV files:")
        print(biomarker_file_map)
        return biomarker_file_map, file_biomarker_map
    
    # Part 4: Apply the regression model to the modified datasets and identify the most significant biomarkers
    def part_4(biomarker_file_map, file_biomarker_map):
        folder_path = output_folder_path
        
        def construct_linear_regression_model(data):
            training_data = data.iloc[:normal_data_size]
            X = training_data[['0-Dim Hole Min Life Range', '0-Dim Hole Max Life Range', 'Median_Death_0D', 'Std_Death_0D']]
            y = training_data['Avg_Death_0D']
            
            model = LinearRegression()
            model.fit(X, y)
            
            return model, model.coef_, model.intercept_
        
        def predict_and_analyze(file_path):
            new_data = pd.read_csv(file_path, header=0)
            _, coefficients, intercept = construct_linear_regression_model(new_data)
            
            def predict_Avg_Death_0D(row):
                return (coefficients[0] * row['0-Dim Hole Min Life Range'] +
                        coefficients[1] * row['0-Dim Hole Max Life Range'] +
                        coefficients[2] * row['Median_Death_0D'] +
                        coefficients[3] * row['Std_Death_0D'] +
                        intercept)
            
            new_data['Predicted Avg_Death_0D'] = new_data.apply(predict_Avg_Death_0D, axis=1)
            new_data['Difference'] = new_data['Avg_Death_0D'] - new_data['Predicted Avg_Death_0D']
            new_data['Regression Difference Percentage'] = abs(new_data['Difference'] / new_data['Avg_Death_0D']) * 100
            
            control_data = new_data.iloc[:normal_data_size]
            reference_point = (control_data['Average Birth (1D)'].mean(), control_data['Average Death (1D)'].mean())
            
            def euclidean_distance(x1, y1, x2, y2):
                return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            
            new_data['Distance %(1)'] = new_data.apply(
                lambda row: euclidean_distance(row['Average Birth (1D)'], row['Average Death (1D)'], reference_point[0], reference_point[1]) * 100,
                axis=1
            )
            
            threshold = 0
            max_threshold = 1.0
            
            while threshold < max_threshold:
                new_data['Result(Cancer)'] = new_data.apply(
                    lambda row: 'Positive' if sum([
                        row['Regression Difference Percentage'] > threshold,
                        row['Distance %(1)'] > Normal_threshold,
                    ]) >= 2 else 'Negative',
                    axis=1
                )
                
                positive_counts = new_data[new_data['Result(Cancer)'] == 'Positive'].groupby('Cohort').size()
                Normal_positive = positive_counts.get('Normal', 0)
                
                if Normal_positive == required_normal_positives:
                    total_positive_cancers = sum(positive_counts.get(t, 0) for t in cancer_types)
                    return total_positive_cancers
                
                threshold += 0.001
            
            return -1
        
        positive_cancer_counts = []
        
        for i in range(1, len(Select_Biomarkers) + 1):
            file_name = f'FeatureVectorCancerDet{i}.csv'
            file_path = os.path.join(folder_path, file_name)
            
            if os.path.exists(file_path):
                result = predict_and_analyze(file_path)
                print(f"Total Positive Cancers for {file_name} is: {result}")
                positive_cancer_counts.append((file_name, result))
            else:
                print(f"File {file_name} not found.")
        
        sorted_positive_cancer_counts = sorted(positive_cancer_counts, key=lambda x: x[1], reverse=True)
        top_files = sorted_positive_cancer_counts[:top_n_files_to_eliminate]
        remaining_files = [file for file in positive_cancer_counts if file not in top_files]
        
        remaining_file_names = [file_name for file_name, _ in remaining_files]
        print(f"\nRemaining CSV Files after eliminating top {top_n_files_to_eliminate} Total Positive Cancers:")
        print(remaining_file_names)
        
        remaining_biomarkers = [biomarker_file_map[file_name] for file_name in remaining_file_names if file_name in biomarker_file_map]
        print(f"The Significant Biomarkers: {remaining_biomarkers}")
        
        eliminated_biomarkers = [biomarker_file_map[file_name] for file_name, _ in top_files if file_name in biomarker_file_map]
        print(f"Eliminated least significant {top_n_files_to_eliminate} biomarkers: {eliminated_biomarkers}")
    
    # Execute the parts in sequence
    part_1()
    Normal_threshold = part_2()
    biomarker_file_map, file_biomarker_map = part_3()
    part_4(biomarker_file_map, file_biomarker_map)

# Run the function with user-defined values
Optimized_Biomarker_Selection(
    distance_threshold_offset=0.8,
    tumor_types={'normal': 554, 'breast': 156, 'colorectum': 291, 'esophagus': 34, 'liver': 33, 'lung': 78, 'ovary': 40, 'pancreas': 69, 'stomach': 50},
    max_patients=1802,
    normal_data_size=554,
    top_n_files_to_eliminate=0,
    cancer_types=['Breast', 'Colorectum', 'Esophagus', 'Liver', 'Lung', 'Ovary', 'Pancreas', 'Stomach'],
    Select_Biomarkers=['Angiopoietin-2', 'CA-125', 'CA 15-3', 'CEA', 'CYFRA 21-1', 'FGF2', 'G-CSF', 'HE4', 'HGF', 'PAR', 'sPECAM-1', 'Thrombospondin-2'],
    clinical_data_path=r'D:\Cancer Detection folder\Biomarker Selection\Clinical cancer data.xlsx',
    sheet_name = 'Normal and Cancer',
    feature_vector_path="D:/Cancer Detection folder/Biomarker Selection/FeatureVectorCancerDet.csv",
    output_folder_path="D:/Cancer Detection folder/Biomarker Selection",
    required_normal_positives=36  # User-defined required number of normal positives
)

CSV file 'FeatureVectorCancerDet.csv' updated with rows P_60 to P_1802.
CSV file 'D:/Cancer Detection folder/Biomarker Selection/FeatureVectorCancerDet.csv' has been updated with all the patient information.

Regression Equation: Avg_Death_0D = 0.1240 * 0-Dim Hole Min Life Range + 0.1898 * 0-Dim Hole Max Life Range + 0.1704 * Median_Death_0D + -0.3762 * Std_Death_0D + 0.3394

Reference Point (Normal Cohort): (0.7140547364620938, 0.757049797833935)

Updated data has been saved back to the CSV file.

Final Averages Data:
       Cohort  0-Dim Hole Min Life Range  Range_Death_0D  \
0      Normal                   0.231512        0.567149   
1      Breast                   0.230688        0.565749   
2  Colorectum                   0.233316        0.565936   
3   Esophagus                   0.226767        0.566153   
4       Liver                   0.229667        0.560179   
5        Lung                   0.236201        0.561862   
6       Ovary                   0.206019        0.60410

In [3]:
# Determination of the final parameters for the cancer detection model

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def get_thresholds(
    feature_vector_path,  # Path to the feature vector CSV file
    normal_data_size,     # Number of normal patients in the dataset
    distance_threshold_offset,  # Offset for the distance threshold
    cancer_types,         # List of cancer types to calculate total positive cancers
    tumor_types,          # Dictionary of tumor types and their row limits
    required_normal_positives  # User-defined required number of normal positives
):
    """
    Calculate and return the thresholds:
    - Normal_distance_threshold (k_2)
    - Regression Difference Threshold (k_1)
    - Total Positive Cancers
    """
    # Load the feature vector data
    new_data = pd.read_csv(feature_vector_path, header=0)
    
    def construct_linear_regression_model():
        # Prepare the data for regression
        data = new_data.iloc[:normal_data_size]
        X = data[['0-Dim Hole Min Life Range', '0-Dim Hole Max Life Range', 'Median_Death_0D', 'Std_Death_0D']]
        y = data['Avg_Death_0D']
        
        # Fit the regression model
        model = LinearRegression()
        model.fit(X, y)
        
        # Get coefficients and intercept
        coefficients = model.coef_
        intercept = model.intercept_
        
        print(f"\nRegression Equation: Avg_Death_0D = {coefficients[0]:.4f} * 0-Dim Hole Min Life Range + "
              f"{coefficients[1]:.4f} * 0-Dim Hole Max Life Range + "
              f"{coefficients[2]:.4f} * Median_Death_0D + "
              f"{coefficients[3]:.4f} * Std_Death_0D + "
              f"{intercept:.4f}")
        
        return model, coefficients, intercept
    
    def predict_and_analyze():
        # Construct the regression model
        _, coefficients, intercept = construct_linear_regression_model()
        
        # Predict Avg_Death_0D for all rows
        def predict_Avg_Death_0D(row):
            return (coefficients[0] * row['0-Dim Hole Min Life Range'] +
                    coefficients[1] * row['0-Dim Hole Max Life Range'] +
                    coefficients[2] * row['Median_Death_0D'] +
                    coefficients[3] * row['Std_Death_0D'] +
                    intercept)
        
        new_data['Predicted Avg_Death_0D'] = new_data.apply(predict_Avg_Death_0D, axis=1)
        new_data['Difference'] = new_data['Avg_Death_0D'] - new_data['Predicted Avg_Death_0D']
        new_data['Regression Difference Percentage'] = abs(new_data['Difference'] / new_data['Avg_Death_0D']) * 100
        
        # Calculate the reference point for the normal cohort
        Normal_data = new_data.iloc[:normal_data_size]
        reference_point = (Normal_data['Average Birth (1D)'].mean(), Normal_data['Average Death (1D)'].mean())
        print(f"\nReference Point (Normal Cohort): {reference_point}")
        
        # Calculate Euclidean distance for each row
        def euclidean_distance(x1, y1, x2, y2):
            return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        
        new_data['Distance %(1)'] = new_data.apply(
            lambda row: euclidean_distance(row['Average Birth (1D)'], row['Average Death (1D)'], reference_point[0], reference_point[1]) * 100,
            axis=1
        )
        
        # Calculate the Normal_distance_threshold (k_2)
        columns_to_average = [
            '0-Dim Hole Min Life Range',
            'Range_Death_0D',
            '1-Dim Hole Max Life Range',
            'Average Birth (1D)',
            'Average Death (1D)',
            'Regression Difference Percentage',
            'Distance %(1)'
        ]
        
        averages_list = []
        
        for tumor, row_limit in tumor_types.items():
            filtered_data = new_data[new_data['Cohort'].str.strip().str.lower() == tumor]
            selected_data = filtered_data.iloc[:row_limit]
            selected_data = selected_data[columns_to_average].dropna()
            column_averages = selected_data.mean(axis=0)
            averages_list.append({'Cohort': tumor.capitalize(), **column_averages.to_dict()})
        
        averages_data = pd.DataFrame(averages_list)
        # print("\nFinal Averages Data:")
        # print(averages_data)
        
        Normal_row = averages_data[averages_data['Cohort'].str.strip().str.lower() == 'normal']
        
        if not Normal_row.empty:
            Normal_distance_threshold = Normal_row['Distance %(1)'].values[0] + distance_threshold_offset
            print(f"\nFinal 'Distance %(1)' Threshold (k_2): {Normal_distance_threshold:.6f}")
        else:
            print("Error: 'Normal' cohort not found in averages_data.")
            return None, None, None, None
        
        # Calculate the Regression Difference Threshold (k_1)
        threshold = 0
        max_threshold = 1.0
        
        while threshold < max_threshold:
            new_data['Result(Cancer)'] = new_data.apply(
                lambda row: 'Positive' if sum([
                    row['Regression Difference Percentage'] > threshold,
                    row['Distance %(1)'] > Normal_distance_threshold
                ]) >= 2 else 'Negative',
                axis=1
            )
            
            positive_counts = new_data[new_data['Result(Cancer)'] == 'Positive'].groupby('Cohort').size()
            Normal_positive = positive_counts.get('Normal', 0)
            
            if Normal_positive == required_normal_positives:
                print(f"\nFinal Regression Difference Threshold (k_1): {threshold:.4f}")
                total_positive_cancers = sum(positive_counts.get(t, 0) for t in cancer_types)
                print(f"Total Positive Cancers: {total_positive_cancers}")
                # Return the Normal_distance_threshold, regression_threshold, total_positive_cancers, and 1-Dim Hole Max Life Range
                return Normal_distance_threshold, threshold, total_positive_cancers, Normal_row['1-Dim Hole Max Life Range'].values[0]
            
            threshold += 0.001
        
        print("\nRequired result for the given threshold is not obtained.")
        return Normal_distance_threshold, None, None, None
    
    # Call the predict_and_analyze function to get the thresholds
    Normal_distance_threshold, regression_threshold, total_positive_cancers, normal_1_dim_hole_max_life_range = predict_and_analyze()
    
    return Normal_distance_threshold, regression_threshold, total_positive_cancers, normal_1_dim_hole_max_life_range

# Example usage
feature_vector_path = "D:/Cancer Detection folder/Biomarker Selection/FeatureVectorCancerDet.csv"
normal_data_size = 554
cancer_types = ['Breast', 'Colorectum', 'Esophagus', 'Liver', 'Lung', 'Ovary', 'Pancreas', 'Stomach']
tumor_types = {'normal': 554}

# Initialize variables to store the best result
best_total_positive_cancers = 0
best_Normal_distance_threshold = 0
best_regression_threshold = 0
best_normal_1_dim_hole_max_life_range = 0

# User-defined required number of normal positives
required_normal_positives = 36  # Example: User-defined value

# Loop through distance_threshold_offset from -1 to 2 with an increment of 0.1
for distance_threshold_offset in np.arange(0, 2, 0.1):
    print(f"\nCalculating thresholds for distance_threshold_offset = {distance_threshold_offset:.1f}")
    Normal_distance_threshold, regression_threshold, total_positive_cancers, normal_1_dim_hole_max_life_range = get_thresholds(
        feature_vector_path, normal_data_size, distance_threshold_offset, cancer_types, tumor_types, required_normal_positives
    )
    
    if total_positive_cancers is not None and total_positive_cancers > best_total_positive_cancers:
        best_total_positive_cancers = total_positive_cancers
        best_Normal_distance_threshold = Normal_distance_threshold
        best_regression_threshold = regression_threshold
        best_normal_1_dim_hole_max_life_range = normal_1_dim_hole_max_life_range

print(f"\nBest Result:")
print(f"Highest Total Positive Cancers: {best_total_positive_cancers}")
print(f"Normal Distance Threshold (k_2): {best_Normal_distance_threshold}")
print(f"Regression Difference Threshold (k_1): {best_regression_threshold}")
print(f"1-Dim Hole Max Life Range (Normal Cohort): {best_normal_1_dim_hole_max_life_range}")


Calculating thresholds for distance_threshold_offset = 0.0

Regression Equation: Avg_Death_0D = 0.1240 * 0-Dim Hole Min Life Range + 0.1898 * 0-Dim Hole Max Life Range + 0.1704 * Median_Death_0D + -0.3762 * Std_Death_0D + 0.3394

Reference Point (Normal Cohort): (0.7140547364620938, 0.757049797833935)

Final 'Distance %(1)' Threshold (k_2): 1.232344

Final Regression Difference Threshold (k_1): 0.8100
Total Positive Cancers: 437

Calculating thresholds for distance_threshold_offset = 0.1

Regression Equation: Avg_Death_0D = 0.1240 * 0-Dim Hole Min Life Range + 0.1898 * 0-Dim Hole Max Life Range + 0.1704 * Median_Death_0D + -0.3762 * Std_Death_0D + 0.3394

Reference Point (Normal Cohort): (0.7140547364620938, 0.757049797833935)

Final 'Distance %(1)' Threshold (k_2): 1.332344

Final Regression Difference Threshold (k_1): 0.7700
Total Positive Cancers: 438

Calculating thresholds for distance_threshold_offset = 0.2

Regression Equation: Avg_Death_0D = 0.1240 * 0-Dim Hole Min Life Range 

In [7]:
# Verification of the number of healthy baseline samples used in the model

In [8]:
import pandas as pd
import numpy as np
import gudhi
import os
from sklearn.linear_model import LinearRegression

# Function to generate FeatureVectorCancerDet.csv for a given normal_data_size
def generate_feature_vector(normal_data_size, file_path, sheet_name, columns_to_select, output_csv_path):
    # Load the clinical data
    df_1 = pd.read_excel(file_path, sheet_name=sheet_name)
    df_2 = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Select the biomarkers
    df_1 = df_1[columns_to_select]
    df_2 = df_2[columns_to_select]
    
    # Select rows 0 to normal_data_size-1 from df_1 to create a fixed dataset for normal patients
    df_1 = df_1.iloc[:normal_data_size]
    
    # Initialize a list to store the final topological features
    data_list = []
    
    # Loop through rows normal_data_size to 1803 of df_2 to include all patients in the analysis
    for index in range(normal_data_size+1, 1803):
        # Select a specific row from df_2
        row_to_add = df_2.iloc[[index]]  # Keep it as a DataFrame
        
        # Append this row to df_1 using pd.concat
        df_new = pd.concat([df_1] + [row_to_add] * 1, ignore_index=True)
        
        # Log normalize the combined data
        numeric_cols = df_new.select_dtypes(include=[np.number]).columns
        df_new[numeric_cols] = np.log(df_new[numeric_cols])
        
        # Save column names for later use
        column_names = df_new.columns.tolist()
        
        # Convert DataFrame to numpy array without column names
        df_subset = df_new.values
        
        # Calculate Pearson correlation matrix
        corr_matrix = np.corrcoef(df_subset, rowvar=False)
        corr_matrix_df = pd.DataFrame(corr_matrix, columns=column_names, index=column_names)
        
        # Convert correlation matrix to a distance matrix
        distance_matrix = 1 - np.abs(corr_matrix_df)
        distance_matrix_np = distance_matrix.values
        
        # Construct a simplicial complex using Gudhi's RipsComplex
        rips_complex = gudhi.RipsComplex(distance_matrix=distance_matrix_np)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
        
        # Compute persistence homology
        persistence = simplex_tree.persistence()
        
        # Extract the topological features
        finite_persistence = [(dim, (birth, death)) for dim, (birth, death) in persistence if death != float('inf')]
        one_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 1]
        if one_dim_holes:
            max_life_range_1D = max(death - birth for birth, death in one_dim_holes)
            avg_birth_1D = np.mean([birth for birth, _ in one_dim_holes])
            avg_death_1D = np.mean([death for _, death in one_dim_holes])
            num_one_dim_holes = len(one_dim_holes)
        else:
            max_life_range_1D = avg_birth_1D = avg_death_1D = num_one_dim_holes = 0
        
        zero_dim_holes = [(birth, death) for dim, (birth, death) in finite_persistence if dim == 0]
        min_life_range_0D = min(death - birth for birth, death in zero_dim_holes)
        max_life_range_0D = max(death - birth for birth, death in zero_dim_holes)
        avg_death_0D = np.mean([death for _, death in zero_dim_holes])
        num_zero_dim_holes = len(zero_dim_holes) + 1
        range_death_0D = max([death for _, death in zero_dim_holes]) - min([death for _, death in zero_dim_holes])
        median_death_0D = np.median([death for _, death in zero_dim_holes])
        std_death_0D = np.std([death for _, death in zero_dim_holes])
        
        # Prepare feature vector data
        data = {
            "0-Dim Hole Min Life Range": [min_life_range_0D],
            "0-Dim Hole Max Life Range": [max_life_range_0D],
            "Range_Death_0D": [range_death_0D],
            "Num_0D_Holes": [num_zero_dim_holes],
            "Median_Death_0D": [median_death_0D],
            "Std_Death_0D": [std_death_0D],
            "Avg_Death_0D": [avg_death_0D],
            "Average Birth (1D)": [avg_birth_1D],
            "Average Death (1D)": [avg_death_1D],
            "1-Dim Hole Max Life Range": [max_life_range_1D],
            "Number of 1-Dimensional Holes": [num_one_dim_holes]
        }
        
        # Create a DataFrame for the current iteration
        df = pd.DataFrame(data, index=[f"P_{index}"])
        
        # Append the DataFrame to the list
        data_list.append(df)
    
    # Concatenate all DataFrames
    final_df = pd.concat(data_list)
    
    # Add Patient ID, Sample ID, Cohort, and AJCC Stage to the final CSV
    additional_columns = ['Patient ID #', 'Cohort', 'AJCC Stage']
    df_clinical = pd.read_excel(file_path, sheet_name=sheet_name)
    df_additional_info = df_clinical[additional_columns].iloc[normal_data_size+1:1803]  # Corresponding rows to additional patients
    additional_info_index = [f"P_{i}" for i in range(normal_data_size+1, 1803)]
    df_additional_info.index = additional_info_index
    
    # Merge the additional columns with the TDA results
    df_combined = pd.concat([df_additional_info, final_df], axis=1)
    
    # Save the updated DataFrame to the CSV file
    df_combined.to_csv(output_csv_path, float_format="%f", index=True)
    print(f"CSV file '{output_csv_path}' generated for normal_data_size = {normal_data_size}.")

# Function to calculate thresholds and total_positive_cancers
def get_thresholds(
    feature_vector_path,  # Path to the feature vector CSV file
    normal_data_size,     # Number of normal patients in the dataset
    distance_threshold_offset,  # Offset for the distance threshold
    cancer_types,         # List of cancer types to calculate total positive cancers
    tumor_types,          # Dictionary of tumor types and their row limits
    required_normal_positives  # User-defined required number of normal positives
):
    """
    Calculate and return the thresholds:
    - Normal_distance_threshold (k_2)
    - Regression Difference Threshold (k_1)
    - Total Positive Cancers
    """
    # Load the feature vector data
    new_data = pd.read_csv(feature_vector_path, header=0)
    
    def construct_linear_regression_model():
        # Prepare the data for regression
        data = new_data.iloc[:normal_data_size]
        X = data[['0-Dim Hole Min Life Range', '0-Dim Hole Max Life Range', 'Median_Death_0D', 'Std_Death_0D']]
        y = data['Avg_Death_0D']
        
        # Fit the regression model
        model = LinearRegression()
        model.fit(X, y)
        
        # Get coefficients and intercept
        coefficients = model.coef_
        intercept = model.intercept_
        
        return model, coefficients, intercept
    
    def predict_and_analyze():
        # Construct the regression model
        _, coefficients, intercept = construct_linear_regression_model()
        
        # Predict Avg_Death_0D for all rows
        def predict_Avg_Death_0D(row):
            return (coefficients[0] * row['0-Dim Hole Min Life Range'] +
                    coefficients[1] * row['0-Dim Hole Max Life Range'] +
                    coefficients[2] * row['Median_Death_0D'] +
                    coefficients[3] * row['Std_Death_0D'] +
                    intercept)
        
        new_data['Predicted Avg_Death_0D'] = new_data.apply(predict_Avg_Death_0D, axis=1)
        new_data['Difference'] = new_data['Avg_Death_0D'] - new_data['Predicted Avg_Death_0D']
        new_data['Regression Difference Percentage'] = abs(new_data['Difference'] / new_data['Avg_Death_0D']) * 100
        
        # Calculate the reference point for the normal cohort
        Normal_data = new_data.iloc[:normal_data_size]
        reference_point = (Normal_data['Average Birth (1D)'].mean(), Normal_data['Average Death (1D)'].mean())
        
        # Calculate Euclidean distance for each row
        def euclidean_distance(x1, y1, x2, y2):
            return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        
        new_data['Distance %(1)'] = new_data.apply(
            lambda row: euclidean_distance(row['Average Birth (1D)'], row['Average Death (1D)'], reference_point[0], reference_point[1]) * 100,
            axis=1
        )
        
        # Calculate the Normal_distance_threshold (k_2)
        columns_to_average = [
            '0-Dim Hole Min Life Range',
            'Range_Death_0D',
            '1-Dim Hole Max Life Range',
            'Average Birth (1D)',
            'Average Death (1D)',
            'Regression Difference Percentage',
            'Distance %(1)'
        ]
        
        averages_list = []
        
        for tumor, row_limit in tumor_types.items():
            filtered_data = new_data[new_data['Cohort'].str.strip().str.lower() == tumor]
            selected_data = filtered_data.iloc[:row_limit]
            selected_data = selected_data[columns_to_average].dropna()
            column_averages = selected_data.mean(axis=0)
            averages_list.append({'Cohort': tumor.capitalize(), **column_averages.to_dict()})
        
        averages_data = pd.DataFrame(averages_list)
        
        Normal_row = averages_data[averages_data['Cohort'].str.strip().str.lower() == 'normal']
        
        if not Normal_row.empty:
            Normal_distance_threshold = Normal_row['Distance %(1)'].values[0] + distance_threshold_offset
        else:
            return None, None, None, None
        
        # Calculate the Regression Difference Threshold (k_1)
        threshold = 0
        max_threshold = 1.0
        
        while threshold < max_threshold:
            new_data['Result(Cancer)'] = new_data.apply(
                lambda row: 'Positive' if sum([
                    row['Regression Difference Percentage'] > threshold,
                    row['Distance %(1)'] > Normal_distance_threshold
                ]) >= 2 else 'Negative',
                axis=1
            )
            
            positive_counts = new_data[new_data['Result(Cancer)'] == 'Positive'].groupby('Cohort').size()
            Normal_positive = positive_counts.get('Normal', 0)
            
            if Normal_positive == required_normal_positives:
                total_positive_cancers = sum(positive_counts.get(t, 0) for t in cancer_types)
                return Normal_distance_threshold, threshold, total_positive_cancers, Normal_row['1-Dim Hole Max Life Range'].values[0]
            
            threshold += 0.001
        
        return Normal_distance_threshold, None, None, None
    
    # Call the predict_and_analyze function to get the thresholds
    Normal_distance_threshold, regression_threshold, total_positive_cancers, normal_1_dim_hole_max_life_range = predict_and_analyze()
    
    return total_positive_cancers

# Function to calculate the highest total_positive_cancers for a given normal_data_size
def calculate_highest_total_positive_cancers(normal_data_size, feature_vector_path, cancer_types, tumor_types, required_normal_positives):
    # Initialize variables to store the best result
    best_total_positive_cancers = 0

    # Loop through distance_threshold_offset from -0.5 to 0.5 with an increment of 0.1
    for distance_threshold_offset in np.arange(-0.1, 1.1, 0.1):
        total_positive_cancers = get_thresholds(
            feature_vector_path, normal_data_size, distance_threshold_offset, cancer_types, tumor_types, required_normal_positives
        )
        
        if total_positive_cancers is not None and total_positive_cancers > best_total_positive_cancers:
            best_total_positive_cancers = total_positive_cancers

    return best_total_positive_cancers

# Main function to loop through normal_data_size and calculate highest total_positive_cancers
def Check_baseline_sample():
    # User-defined variables
    file_path = r'D:\Cancer Detection folder\Baseline\Clinical cancer data.xlsx'
    sheet_name = 'Normal and Cancer'
    columns_to_select = ['AFP', 'Angiopoietin-2', 'AXL', 'CA-125', 'CA 15-3', 'CA19-9', 'CD44', 'CEA', 'CYFRA 21-1', 'DKK1', 'Endoglin', 'FGF2', 'Follistatin', 'Galectin-3', 'G-CSF', 'GDF15', 'HE4', 'HGF', 'IL-6', 'IL-8', 'Kallikrein-6', 'Leptin', 'Mesothelin', 'Midkine', 'Myeloperoxidase', 'NSE', 'OPG', 'OPN', 'PAR', 'Prolactin', 'sEGFR', 'sFas', 'SHBG', 'sHER2/sEGFR2/sErbB2', 'sPECAM-1', 'TGFa', 'Thrombospondin-2', 'TIMP-1', 'TIMP-2']
    output_csv_path = "D:/Cancer Detection folder/Baseline/FeatureVectorCancerDet.csv"
    cancer_types = ['Breast', 'Colorectum', 'Esophagus', 'Liver', 'Lung', 'Ovary', 'Pancreas', 'Stomach']
    tumor_types = {'normal': 554}
    required_normal_positives = 36  # User-defined value

    # Loop through normal_data_size from 29 to 79 with increments of 10
    for normal_data_size in range(39, 80, 10):
        # Generate FeatureVectorCancerDet.csv for the current normal_data_size
        generate_feature_vector(normal_data_size, file_path, sheet_name, columns_to_select, output_csv_path)
        
        # Calculate the highest total_positive_cancers for the current normal_data_size
        highest_total_positive_cancers = calculate_highest_total_positive_cancers(
            normal_data_size, output_csv_path, cancer_types, tumor_types, required_normal_positives
        )
        
        print(f"Highest Total Positive Cancers for normal_data_size = {normal_data_size}: {highest_total_positive_cancers}")

# Run the main function
if __name__ == "__main__":
    Check_baseline_sample()

C:\Users\Sudarshan Gogoi\anaconda3\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Sudarshan Gogoi\anaconda3\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


CSV file 'D:/Cancer Detection folder/Baseline/FeatureVectorCancerDet.csv' generated for normal_data_size = 39.
Highest Total Positive Cancers for normal_data_size = 39: 0
CSV file 'D:/Cancer Detection folder/Baseline/FeatureVectorCancerDet.csv' generated for normal_data_size = 49.
Highest Total Positive Cancers for normal_data_size = 49: 0
CSV file 'D:/Cancer Detection folder/Baseline/FeatureVectorCancerDet.csv' generated for normal_data_size = 59.
Highest Total Positive Cancers for normal_data_size = 59: 117
CSV file 'D:/Cancer Detection folder/Baseline/FeatureVectorCancerDet.csv' generated for normal_data_size = 69.
Highest Total Positive Cancers for normal_data_size = 69: 144
CSV file 'D:/Cancer Detection folder/Baseline/FeatureVectorCancerDet.csv' generated for normal_data_size = 79.
Highest Total Positive Cancers for normal_data_size = 79: 183
